# Tile Generation Benchmarks using CMR

## Explanation

In this notebook we explore tile generation using titiler-cmr with the GPM IMERG and MUR-SST datasets.

In [ ]:
import cf_xarray  # noqa
import hvplot.pandas  # noqa
from cmr_tile_test import run_cmr_tile_tests

import helpers.dataframe as dataframe_helpers
import helpers.eodc_hub_role as eodc_hub_role

## Set parameters for benchmarks

In [ ]:
gpm_imerg = {
    "concept_id": "C2723754850-GES_DISC",
    "variable": "precipitation",
    "cmr_query": {"temporal": ("2000-06-01", "2000-06-01")},
    "rescale": ((0, 48),),
    "colormap_name": "coolwarm",
    "output_format": "png",
}
mur_sst = {
    "concept_id": "C1996881146-POCLOUD",
    "variable": "analysed_sst",
    "cmr_query": {"temporal": ("2002-06-01", "2002-06-01")},
    "rescale": ((271, 305),),
    "colormap_name": "coolwarm",
    "output_format": "png",
}
iterations = 10

## Run the tests

In [ ]:
cmr_test_results = []
cmr_test_results.append(
    run_cmr_tile_tests(
        gpm_imerg,
        iterations,
    )
)
cmr_test_results.append(run_cmr_tile_tests(mur_sst, iterations))

## Load + plot results 

In [ ]:
credentials = eodc_hub_role.fetch_and_set_credentials()
df = dataframe_helpers.load_all_into_dataframe(
    credentials, cmr_test_results, use_boto3=False
)
df = dataframe_helpers.expand_timings(df)
df["Time (s)"] = df["time"] * 1e-3

In [ ]:
plt_opts = {"width": 1800, "height": 400, "xrotation": 90}
df.hvplot.box(
    y="Time (s)",
    by=["zoom", "dataset_id"],
    ylabel="Time to tile (s)",
    xlabel="Zoom level",
    legend=False,
).opts(**plt_opts)